pip install tiktoken

In [3]:
import tiktoken
with open("Data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [4]:
print(raw_text[:1])
tokenizer = tiktoken.get_encoding('gpt2')
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))
enc_sample = enc_text[:50]

I
5147


In [5]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:     {y}")

x: [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


for i in range (1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f"in interation {i}, with : {tokenizer.decode(context)} ==> : {tokenizer.decode([desired])}")

USE A DATALODER AND CREATE A SLIDING WINDOW

In [23]:
from torch.utils.data import Dataset, DataLoader
class GPTDataSetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk)) 

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]


THIS IS THE ACTUAL DATALOADER BELOW :

In [17]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=6):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataSetV1(txt, tokenizer, max_length, stride)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

IMPORTIN AND TRYN AND SHIII

In [39]:
with open("Data.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [35]:
import torch
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
second_batch = next(data_iter)
print("First Iter : ", first_batch)
print("Second Iter : ", second_batch)

First Iter :  [tensor([[  40,  220,  367, 2885]]), tensor([[ 220,  367, 2885, 1464]])]
Second Iter :  [tensor([[ 220,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


CREATING  POSITIONAL TOKEN EMBEDINGS

In [36]:
vocab_size= 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


Instantiate the dataloader to sample data amd create a batch size 8 , so resulting in a 8 x 4 x 256 tensor

In [44]:
max_length =4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)

data_iter = iter(dataloader)
inputs,targets = next(data_iter)
#for i in range(5):
 #   print("\nInput tokens : \n", tokenizer.decode(inputs[i].tolist()))
    #print("\n Inputs Shape: \n", inputs.shape)\
    
  #  print("\n Targets tokens : \n", tokenizer.decode(targets[i].tolist()))
    #print("\n Targets Shape: \n", targets.shape)

Embed inputs into 256 dimension vector 